In [ ]:
!pip install pandas
!pip install beautifulsoup4
!pip install requests

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import typing
import time
from datetime import datetime, timedelta

# Gas prices

In [2]:
url = f'https://www.peco-online.ro/index.php'

In [2]:
def doGet(url: str) -> BeautifulSoup:
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36'}
    return BeautifulSoup(requests.get(url, allow_redirects=True, headers=headers).content, 'html.parser')

In [3]:
def doPost(url: str, form_data: dict) -> BeautifulSoup:
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36'}
    return BeautifulSoup(requests.post(url, form_data, headers=headers).content, 'html.parser')

In [5]:
page = doGet(url)

In [6]:
# get all gas vendors
vendors = [option.get('value').strip() for option in page.find('select', id='Retea').find_all('option')]

In [7]:
# get all gas types
types = [option.get('value').strip() for option in page.find('select', id='carburant').find_all('option')]

In [6]:
counties_df = pd.read_json('counties.json', orient='records')

In [18]:
def createFormData(vendors: list, types: list, counties: list) -> list:
    form_datas = []

    for county in counties:
        for type in types:
            form_datas.append({
                'carburant': type,
                'locatie': 'Judet',
                'nume_locatie': county,
                'retea[]': vendors
            })

    return form_datas

In [26]:
form_datas = createFormData(vendors, types, counties_df['County'].to_list())
data = []

for form_data in form_datas:
    data.append({'type': form_data['carburant'], 'county': form_data['nume_locatie'], 'data': doPost(url, form_data)})
    time.sleep(0.15)

In [40]:
scraped_data = []

for county_findings in data:
    dom = county_findings['data']
    try:
        findings = dom.find('table', id='tabelaRezultate').find_all('tr')[1:]
        for finding in findings:
            cells = finding.find_all('td')

            price = cells[0].find('span', class_='pret').text.strip()
            vendor = cells[1].find('img').get('title').strip()
            city = cells[1].find('span', class_='small d-block text-muted').text.strip()
            addr = cells[1].find('span', class_='').text.strip() # will be kept only for uniqueness of data

            scraped_data.append({'Type': county_findings['type'], 'County': county_findings['county'], 'City': city, 'Addr': addr, 'Vendor': vendor, 'Price': price})
    except:
        print('Error at ' + county_findings['county'] + ' with type ' + county_findings['type'] + '. Info: \n' + str(dom))

In [42]:
scraped_df = pd.DataFrame(scraped_data)

In [43]:
# delete all falsy values
scraped_df['Price'].replace('- -', np.nan, inplace=True)
scraped_df['Price'] = pd.to_numeric(scraped_df['Price'])
# better type definition
scraped_df['Type'] = [value.replace('_', ' ') for value in scraped_df['Type']]
# delete county names from city names
scraped_df['City'] = [re.sub(',[^,]*$', '', value) for value in scraped_df['City']]
# add timestamp to scrape
scraped_df['Timestamp'] = pd.to_datetime('now')

In [35]:
type_ro_hun_assoc = {
    'Benzina Regular': '95-ös Benzin',
    'GPL': 'LPG',
    'Benzina Premium': 'Prémium Benzin',
    'Motorina Regular': 'Gázolaj',
    'Motorina Premium': 'Prémium Gázolaj',
    'AdBlue': 'AdBlue'
}

def type_translation_to_hun(type: str):
    return type_ro_hun_assoc[type]

In [36]:
scraped_df['Type'] = [type_translation_to_hun(value) for value in scraped_df['Type']]

In [ ]:
scraped_df = scraped_df.merge(counties_df[['County', 'ISO Code']], how='left', left_on='County', right_on='County')
scraped_df.rename(columns={'ISO Code': 'ISO'}, inplace=True)

In [38]:
path = 'gas_tmp-' + str(pd.to_datetime('now').date().day) + '.json'
scraped_df.to_json(path, orient='records')

# Brent crude oil

In [8]:
url = r'https://finance.yahoo.com/quote/BZ%3DF/history?p=BZ%3DF'

In [9]:
page = doGet(url)

In [10]:
table = page.find('table', attrs={'data-test': 'historical-prices'})

In [11]:
body = table.find('tbody')

In [26]:
def get_values_for_a_month(body: any, month: str) -> pd.DataFrame:
    df = pd.DataFrame(data={'Date': [], 'Closing': []})

    for row in body.find_all('tr'):
        tds = row.find_all('td')

        date = tds[0].text

        if month in date:
            open = float(tds[1].text)
            high = float(tds[2].text)
            low = float(tds[3].text)
            close = float(tds[4].text)
            volume = float(tds[6].text.replace(',', ''))

            df = df.append(pd.DataFrame(data=[[date, open, high, low, close, volume]], columns=['Date', 'Open', 'High', 'Low', 'Closing', 'Volume']))
   
    df['Date'] = pd.to_datetime(df['Date'])

    for i in df.select_dtypes('int').columns.values:
        if i == 'Volume':
            continue

        df[i] = df[i].astype(float)

    return df

In [27]:
def get_values_for_date(body: any, searched_date: str) -> pd.DataFrame:
    df = pd.DataFrame(data={'Date': [], 'Closing': []})

    for row in body.find_all('tr'):
        tds = row.find_all('td')

        date = tds[0].text

        if searched_date in date:
            open = float(tds[1].text)
            high = float(tds[2].text)
            low = float(tds[3].text)
            close = float(tds[4].text)
            volume = float(tds[6].text.replace(',', ''))

            df = df.append(pd.DataFrame(data=[[date, open, high, low, close, volume]], columns=['Date', 'Open', 'High', 'Low', 'Closing', 'Volume']))
   
    df['Date'] = pd.to_datetime(df['Date'])

    for i in df.select_dtypes('int').columns.values:
        if i == 'Volume':
            continue

        df[i] = df[i].astype(float)

    return df

In [28]:
# apr = get_values_for_a_month(body, 'Apr')

In [19]:
# may = get_values_for_a_month(body, 'May')
# may = may.reset_index().drop(columns=['index'])
# may = may.loc[1:]

In [40]:
# apr.to_json('../data/crude_apr.json', orient='records')
# may.to_json('../data/crude_may.json', orient='records')

In [ ]:
yesterday_full = (datetime.now() - timedelta(days=1)).strftime('%b %d, %Y')
yesterday_day = (datetime.now() - timedelta(days=1)).strftime('%d')

In [119]:
today = get_values_for_date(body, yesterday_full)

In [124]:
if today.size:
    path = 'crude_tmp-' + yesterday_day + '.json'
    today.to_json(path, orient='records')